<div>
<img src="figures/svtLogo.png"/>
</div>  

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Lab 8 - Elimination of variables, Penalty and SQP methods</h2></center>

$$\newcommand{\bx}{\mathbf{x}}$$
The following problem is given:
\begin{align*}
  \min_{\bx \in \mathbb{R}^2} \;\; & f(\bx)  \\
   \text{s.t.} \;\;& x_1+x_2=8,
\end{align*}

where $f(\bx) = - (x_1^2+x_2^2+4x_1x_2)$.

<u>Task 1</u>: Find the minimum of the function using variable elimination.
Check the second-order sufficient conditions for the unconstrained one-variable problem.

We solve for $x_1$:
$$\begin{align}
x_1 = 8-x_2 \Leftrightarrow \bar{f}(x_2) & = - ((8-x_2)^2+x_2^2+4(8-x_2)x_2) \\
& = 4x_2^2 - 16  x_2 - 64
\end{align}
$$
We then solve $\bar{f}(x_2)$:
$$
\begin{align}
\frac{\partial \bar{f}}{\partial x_2} &= 4x_2 - 16 = 0 \Leftrightarrow x_2 = 4 \\
\frac{\partial \bar{f}}{\partial x_2^2} &= 4 ( > 0 )
\end{align}
$$

The optimal point $x^* = (4, 4)$


Another possibility is to use the following penalty function:
\begin{align*}
  	Q(\bx;\mu)=f(\bx)+\frac{1}{2\mu} (x_1+x_2-8)^2\,,
\end{align*}
with $\mu>0$ being a penalty parameter.
     
<br>
<u>Task 2</u>: Write down the first-order necessary condition of optimality for minimizing $Q$.

first-order necessary condition of optimality is $\nabla Q(\bx; \mu) = 0$
$$\begin{align}
\frac{\partial Q}{\partial x_1} & = -2 x_1 - 4 x_2 + \frac{1}{\mu}(x_1 + x_2 - 8) = 0 \\
& \Leftrightarrow (\frac{1}{\mu}-2)x_1 + (\frac{1}{\mu}-4)x_2 - 8 = 0 \\
\frac{\partial Q}{\partial x_2} &= -2 x_2 - 4 x_1 + \frac{1}{\mu}(x_1 + x_2 - 8) = 0 \\
& \Leftrightarrow (\frac{1}{\mu}-2)x_2 + (\frac{1}{\mu}-4)x_1 - 8 = 0 \\
\end{align}$$

<u>Task 3</u>: What happens as $\mu \rightarrow 0$?  Complete the implementation of the quadratic penalty method below:

Also, report the eigenvalues and the condition number of the Hessian for each $\mu$.

In [1]:
import numpy as np
import scipy.optimize as sp

# to calculate the gradient and Hessian of the objective function
from autograd import grad
from autograd import hessian

# to solve additionally using SLSQP solver, later on
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from math import inf

### Objective, constraint, quadratic penalty function, gradient and hessian

In [2]:
def objective(X):
    x1, x2 = X[0], X[1]
    f = -(x1 ** 2 + x2 ** 2 + 4 * x1 * x2)
    return f

In [3]:
def constraint(X):
    x1, x2 = X[0], X[1]
    c = x1 + x2 - 8
    return c

In [29]:
def penaltyFunction(X, mu):
    f = objective(X) + 1/(2*mu) * constraint(X)**2
    return f

In [23]:
def gradient_function(x, mu): 
    return [el.item() for el in grad(penaltyFunction, 0)(x, mu)]

In [24]:
def hessian_function(x, mu): 
    return hessian(penaltyFunction, 0)(x, mu)

### Quadratic penalty method

In [25]:
def qpm(x0, mu): 
    
    # get eigenvalues of the Hessian
    w, v = np.linalg.eig(hessian_function(x0, mu))
    
    # get condition number of the Hessian
    n = np.linalg.cond(hessian_function(x0, mu))
    
    # unconstrained optimization using BFGS method
    res = sp.minimize(penaltyFunction, x0, args=(mu), method='BFGS', jac=gradient_function)
    
    return w, n, res.x

In [26]:
mu = 1
x0 = np.array([1.,1.])

# acceptable constraint violation at optimum
eps_viol = 1e-15
constraint_violation = True

it = 0

print ("{:<10} {:<10} {:<20}\t{:^20} {:^20}\t{:^30}".format('iter','mu','minimum','condition nr.', 'constraint violation', 'eigenvalues'))
while constraint_violation:
    it = it + 1
    
    w, n, xmin = qpm(x0,mu)
    print ("{:<10d} {:<10.3e} [{:^8.4f}, {:^8.4f}]\t{:<4} {:<20.2e} {:^20.3e}\t[{:^8.4f}, {:^8.4f}]".format(it,mu,xmin[0],xmin[1],' ',n,constraint(xmin), w[0], w[1]))
    
    if constraint(xmin) <= eps_viol:
        constraint_violation = False 
    
    # update for next iteration (e.g. half of previous penalty value)
    x0 = xmin
    mu = mu/2

iter       mu         minimum             	   condition nr.     constraint violation	         eigenvalues          
1          1.000e+00  [732.3181, 732.3181]	     2.00e+00                  1.457e+03      	[ 2.0000 , -4.0000 ]
2          5.000e-01  [1463.6362, 1463.6362]	     1.00e+00                  2.919e+03      	[ 2.0000 , -2.0000 ]
3          2.500e-01  [16.0000 , 16.0000 ]	     1.00e+00                  2.400e+01      	[ 2.0000 ,  2.0000 ]
4          1.250e-01  [ 6.4000 ,  6.4000 ]	     5.00e+00                  4.800e+00      	[10.0000 ,  2.0000 ]
5          6.250e-02  [ 4.9231 ,  4.9231 ]	     1.30e+01                  1.846e+00      	[26.0000 ,  2.0000 ]
6          3.125e-02  [ 4.4138 ,  4.4138 ]	     2.90e+01                  8.276e-01      	[58.0000 ,  2.0000 ]
7          1.562e-02  [ 4.1967 ,  4.1967 ]	     6.10e+01                  3.934e-01      	[122.0000,  2.0000 ]
8          7.812e-03  [ 4.0960 ,  4.0960 ]	     1.25e+02                  1.920e-01      	[250.0000,  2.0

### SLSQP method (scipy)

We will solve the problem using scipy's SLSQP solver (written by Dieter Kraft, DLR Oberpfaffenhofen)

In [21]:
x0 = np.array([0.,0.])

bounds = Bounds([-inf,-inf], [inf,inf])

# The constraint is actually linear, so you can also try a different approach.
# See SLSQP documentation for more details on how to set up linear constraints.
nonlinear_constraints = NonlinearConstraint(constraint, 0, 0)

# use SLSQP
res = sp.minimize(objective, x0,
               constraints=[nonlinear_constraints], bounds=bounds, 
                  options={'disp': True, 'iprint': 4} )

print("minimum = {}".format(res.x))
print("constraint violation = {}".format(constraint(res.x)))
# The problem is a QP which is the reason why the SLSQP method is so fast. 

  NIT    FC           OBJFUN            GNORM
    1     4    -9.600000E+01     2.107342E-08
    2     6    -9.600000E+01     3.394112E+01
Optimization terminated successfully    (Exit mode 0)
            Current function value: -95.99999999999997
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2
minimum = [4. 4.]
constraint violation = -1.7763568394002505e-15


# Rosenbrock function contrained

The original Rosenbrock function does not have constraints, however, we introduce a constraint 
$$x_1^2 + x_2^2 - 1 \leq 0$$

In [37]:
def rosenbrock(x):
    return ((x[0]-1)**2 + 100*(x[1]-x[0]**2)**2)

In [38]:
def rosenbrock_inequality(x): 
    return x[0]**2 + x[1]**2 - 1

In [39]:
def penalty_inequality(x, mu): 
    
    f = rosenbrock(x) + 1/(2*mu) * max(0,rosenbrock_inequality(x))**2
    return f

In [40]:
def qpm_inequality(x0, mu): 
    
    # get eigenvalues of the Hessian
    #w, v = np.linalg.eig(hessian_function(x0, mu))
    
    # get condition number of the Hessian
    #n = np.linalg.cond(hessian_function(x0, mu))
    
    # unconstrained optimization using BFGS method
    res = sp.minimize(penalty_inequality, x0, args=(mu), method='BFGS')# jac=gradient_function)

    return res.x
    #return w, n, res.x

In [41]:
mu = 1
x0 = np.array([0.0, 0.0])

# acceptable constraint violation at optimum
eps_f = 1e-8
sufficient_decrease = True

it = 0

print ("{:<10} {:<10} {:<20} {:^20} {:^30}".format('iter','mu','minimum','condition nr.', 'constraint value'))
while sufficient_decrease:
    it = it + 1
    f_prev = rosenbrock(x0)
    xmin = qpm_inequality(x0, mu)
    print ("{:<10d} {:<10.3e} [{:^8.4f}, {:^8.4f}] {:<4} {:<20.2e} {:^20.3e}".format(it,mu,xmin[0],xmin[1],' ',n, rosenbrock_inequality(xmin)))
    
    if abs(f_prev - rosenbrock(xmin)) <= eps_f:
        sufficient_decrease = False 
    
    # update penalty for next iteration (e.g. half of previous value)
    x0 = xmin
    mu = mu/2


iter       mu         minimum                 condition nr.            constraint value       
1          1.000e+00  [ 0.8135 ,  0.6611 ]      2.55e+16                  9.875e-02      
2          5.000e-01  [ 0.8015 ,  0.6417 ]      2.55e+16                  5.423e-02      
3          2.500e-01  [ 0.7945 ,  0.6304 ]      2.55e+16                  2.861e-02      
4          1.250e-01  [ 0.7906 ,  0.6243 ]      2.55e+16                  1.473e-02      
5          6.250e-02  [ 0.7885 ,  0.6210 ]      2.55e+16                  7.476e-03      
6          3.125e-02  [ 0.7875 ,  0.6194 ]      2.55e+16                  3.767e-03      
7          1.562e-02  [ 0.7870 ,  0.6185 ]      2.55e+16                  1.891e-03      
8          7.812e-03  [ 0.7867 ,  0.6181 ]      2.55e+16                  9.473e-04      
9          3.906e-03  [ 0.7865 ,  0.6179 ]      2.55e+16                  4.741e-04      
10         1.953e-03  [ 0.7865 ,  0.6178 ]      2.55e+16                  2.372e-04      
11   

In [42]:
x0 = np.array([0.,0.])

bounds = Bounds([-inf,-inf], [inf,inf])

nonlinear_constraints = NonlinearConstraint(rosenbrock_inequality, -inf, 0)

# use SLSQP
res = sp.minimize(rosenbrock, x0,
               constraints=[nonlinear_constraints], bounds=bounds, options={'disp': True, 'iprint': 4} )

print("minimum = {}".format(res.x))
print("constraint value = {}".format(rosenbrock_inequality(res.x)))

  NIT    FC           OBJFUN            GNORM
    1     4     1.601001E+03     2.000000E+00
    2     8     3.141943E+03     8.158461E+00
    3    13     1.597029E+00     9.044028E+00
    4    17     4.171509E-01     1.494909E+00
    5    20     9.114047E-01     1.060580E+00
    6    24     5.712767E-01     1.679292E+00
    7    28     2.615425E-01     3.334856E+00
    8    31     1.930068E-01     7.378206E+00
    9    34     1.215000E-01     2.677153E+00
   10    37     7.945503E-02     2.483551E+00
   11    40     4.857427E-02     4.150391E+00
   12    43     4.567073E-02     2.384905E+00
   13    46     4.567481E-02     2.366884E-01
   14    48     4.567481E-02     2.429864E-01
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.045674808595787916
            Iterations: 14
            Function evaluations: 48
            Gradient evaluations: 14
minimum = [0.78641516 0.6176983 ]
constraint value = 1.0186842480663927e-09
